# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
weather_df = pd.read_csv('../output_data/cities.csv')
weather_df

,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed
0,Severo-Kuril'sk,RU,50.68,156.12,1607626190,25.81,90,88,16.51
1,Barrow,US,71.29,-156.79,1607626191,-5.57,90,100,13.11
2,Kamaishi,JP,39.27,141.88,1607626191,41.97,74,0,5.75
3,Bluff,NZ,-46.60,168.33,1607626083,44.01,94,100,12.91
4,Upernavik,GL,72.79,-56.15,1607626191,24.82,97,100,7.29
...,...,...,...,...,...,...,...,...,...
556,Kautokeino,NO,69.01,23.04,1607626280,20.17,97,89,6.93
557,Nikel,RU,69.41,30.22,1607626280,32.00,98,90,13.87
558,Road Town,VG,18.42,-64.62,1607626128,84.20,66,20,6.93
559,Timberlake,US,37.32,-79.26,1607626281,60.01,38,1,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
weather_df = weather_df.fillna(0)
humidity = weather_df["Humidity"].astype(float)


Lat    561
Lng    561
dtype: int64

In [51]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
ideal_weather = weather_df.loc[(weather_df['Max Temp'] >= 85) & (weather_df['Wind Speed'] <= 10) & (weather_df['Cloudiness'] <= 20)].dropna()
ideal_weather

,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed
13,Santa Maria,BR,-29.68,-53.81,1607626193,89.78,31,0,8.43
103,Podor,MR,16.65,-14.96,1607626208,86.00,27,0,2.24
208,San Rafael,AR,-34.62,-68.33,1607625768,99.27,14,8,5.28
335,Anori,BR,-3.77,-61.64,1607626229,85.95,75,13,4.27
480,Mayor Pablo Lagerenza,PY,-19.93,-60.77,1607626266,103.33,17,0,5.70
484,Viedma,AR,-40.81,-63.00,1607625720,91.00,35,13,5.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
hotel_df = ideal_weather.loc[:, ['City' , 'Country' , 'Lat' , 'Lng']]
hotel_df["Hotel"] = ''
hotel_df

,City,Country,Lat,Lng,Hotel
13,Santa Maria,BR,-29.68,-53.81,
103,Podor,MR,16.65,-14.96,
208,San Rafael,AR,-34.62,-68.33,
335,Anori,BR,-3.77,-61.64,
480,Mayor Pablo Lagerenza,PY,-19.93,-60.77,
484,Viedma,AR,-40.81,-63.00,


In [34]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "hotel",  
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    params['location'] = f'{lat},{lng}'
    
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    
hotel_df

Retrieving Results for Index 13: Santa Maria.
Missing field/result... skipping.
Retrieving Results for Index 103: Podor.
Missing field/result... skipping.
Retrieving Results for Index 208: San Rafael.
Missing field/result... skipping.
Retrieving Results for Index 335: Anori.
Missing field/result... skipping.
Retrieving Results for Index 480: Mayor Pablo Lagerenza.
Missing field/result... skipping.
Retrieving Results for Index 484: Viedma.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel
13,Santa Maria,BR,-29.68,-53.81,Altadomo Hotel
103,Podor,MR,16.65,-14.96,La cour du fleuve
208,San Rafael,AR,-34.62,-68.33,Azalea Luxury Lodge
335,Anori,BR,-3.77,-61.64,Hotel Central
480,Mayor Pablo Lagerenza,PY,-19.93,-60.77,
484,Viedma,AR,-40.81,-63.00,


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# Add marker layer ontop of heat map
marked_loc = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marked_loc)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))